In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 18.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 56.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 33.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00:00:0100:01m


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [64]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/home/codespace/.python/current/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [65]:
text = "Create a count down function."
query_result = embeddings.embed_query(text)
query_result[:3]

[0.03126324340701103, 0.030047697946429253, -0.02511913701891899]

In [66]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs = new_db.similarity_search(text)

In [67]:
question = "What are the Machine Learning ALgorithms?"
searchDocs = new_db.similarity_search(question)
print(searchDocs[0].page_content)

machine learning models.\n5. ?Machine Learning: A Probabilistic Perspective? by Kevin P. Murphy: This book presents a comprehensive and modern approach to machine learning, with a focus on probabilistic methods."


#LLM

In [68]:
# Create a tokenizer object by loading the pretrained "Intel/dynamic_tinybert" tokenizer.
tokenizer = AutoTokenizer.from_pretrained("Intel/dynamic_tinybert")

# Create a question-answering model object by loading the pretrained "Intel/dynamic_tinybert" model.
model = AutoModelForQuestionAnswering.from_pretrained("Intel/dynamic_tinybert")

In [69]:
# Specify the model name you want to use
model_name = "Intel/dynamic_tinybert"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering", 
    model=model_name, 
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

#Retrieval

In [70]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or documents from the database.
retriever = new_db.as_retriever()

In [71]:
docs = retriever.get_relevant_documents("Give me a Machine Learning Algorithm")
print(docs[0].page_content)

machine learning models.\n5. ?Machine Learning: A Probabilistic Perspective? by Kevin P. Murphy: This book presents a comprehensive and modern approach to machine learning, with a focus on probabilistic methods."


#Retrieval QA Chain

In [72]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = new_db.as_retriever(search_kwargs={"k": 15}, search_type="mmr")

In [73]:
QA_prompt_template = """Your main role is to answer questions from the user. DO NOT CREATE QUESTIONS, ANSWER THEM.
Retrieve relevant information from the dataset and utilize inference and suggestions for the following tasks: document summarization, code/workflow suggestion, code development, code enhancement, and providing up-to-date information on computer science.

**Document Summarization:**
Summarize the provided document(s) based on the given context.

**Code/Workflow Suggestion:**
Suggest a code snippet or workflow that closely aligns with the given context and requirements.

**Code Development:**
Develop a code solution based on the given problem statement and context.

**Code Enhancement:**
Provide suggestions or improvements to enhance the provided code, considering the given context.

**Up-to-date Computer Science Information:**
Provide the most recent and relevant information on computer science topics, considering the given context.

Ensure that the responses are as recent as possible, derived from the dataset, and make use of inference and suggestions to provide comprehensive and useful answers.
Answer the following question based on the given context.

Question: {question}
Context: {context}
Answer:
"""

In [74]:
# Create a question-answering instance (qa) using the RetrievalQA class.
# Creating the Prompt Template
'''
review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["context"], template=QA_prompt_template)
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [review_system_prompt, review_human_prompt]

review_prompt = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)
'''

'\nreview_system_prompt = SystemMessagePromptTemplate(\n    prompt=PromptTemplate(input_variables=["context"], template=QA_prompt_template)\n)\n\nreview_human_prompt = HumanMessagePromptTemplate(\n    prompt=PromptTemplate(input_variables=["question"], template="{question}")\n)\nmessages = [review_system_prompt, review_human_prompt]\n\nreview_prompt = ChatPromptTemplate(\n    input_variables=["context", "question"], messages=messages\n)\n'

In [75]:

# Create a question-answering instance (qa) using the RetrievalQA class.
# Creating the Prompt Template
QA_PROMPT = PromptTemplate(template=QA_prompt_template, input_variables=["context", "question"])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=False,
    chain_type_kwargs={"prompt":QA_PROMPT}
    )



In [85]:
rag_chain_from_docs = (
    {"context":(lambda x: format_docs(x["context"])),"question": RunnablePassthrough()}
    | QA_PROMPT
    | llm
    | StrOutputParser()
)

def RAG_gen(question:str) -> str:
    return rag_chain_from_docs.invoke(question)

In [ ]:
# Create and modify the QA response
def QA_gen(question: str) -> str:
    result = qa_chain.run({"query": question})
    return result["result"]

In [ ]:
qn = "What is hypervisor?"
QA_gen(qn)

In [86]:
qn= "What is Full-Stack?"
RAG_gen(qn)

TypeError: string indices must be integers